In [ ]:
import numpy as np
import h5py
from scipy.io import loadmat

from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model
from sklearn import metrics
from distributed import Executor

import matplotlib.pyplot as plt
# from ipywidgets import interact
% matplotlib inline

In [ ]:
# filebase = '/Users/elyall/Dropbox/Data/7737_291_000'
filebase = '/media/elyall/Data/7737/180118/7737_291_000'
# filebase = '/media/elyall/Data/7142/170710/7142_220_002'
depth = [1,2,3,4];

expfile = filebase + '.exp'
fn = [filebase + '.rois']
# fn = [filebase + "_depth%01d" % d + '.rois' for d in depth]
print(expfile)
print(fn)

number_of_neurons = np.logspace(0,2.5,13).round().astype('int') # list of number of neurons to sample
n_iter = 5     # number of replicates for N samples
n_splits = 5   # cross-validation
max_iter = 100 # regression convergence

print(number_of_neurons)

In [ ]:
# Load stim info
myfile = h5py.File(expfile,'r')
StimID = myfile['/TrialInfo/StimID'].value[:].astype('int')
TrialIndex = myfile['TrialIndex'].value[:].astype('int') - 1
myfile.close()

# Load neural data
temp = []
for f in fn:
    myfile = h5py.File(f,'r')
    numNeurons = len(myfile['ROIdata/rois/stimMean'])
    numTrials = len(myfile['ROIdata/DataInfo/StimID'][0])
    data = np.zeros([numTrials,numNeurons])
    for n in np.arange(numNeurons):
        data[:,n] = myfile[myfile['ROIdata/rois/stimMean'][n][0]].value
    myfile.close()
    temp.append(data)
data = np.concatenate(temp[:],axis=1)

# keep only desired trials
StimID = np.squeeze(StimID[TrialIndex])
data = np.squeeze(data[TrialIndex,:])
# data = np.random.random([np.size(TrialIndex),100]) # fake data

numTrials, numNeurons = np.shape(data)
numX = np.size(number_of_neurons)
print('%01d trials with %01d neurons while randomly sampling the following neurons:' % (numTrials,numNeurons), number_of_neurons)

In [ ]:
def computeLogReg(number_of_neurons):
    
    pred = np.zeros([numTrials,n_iter])
    perc_correct = np.zeros([n_iter])
    
    for ind in np.arange(n_iter):

        # pull out sample of neurons
        my_sample = np.random.choice(range(numNeurons), number_of_neurons)
        current = data[:,my_sample]

        # fit Logistic Regression to random sample (using KFold cross-validation)
        skf = StratifiedKFold(n_splits=n_splits)
        for train_index, test_index in skf.split(current, StimID):
            X_train, X_test = current[train_index,:], current[test_index,:]
            y_train, y_test = StimID[train_index], StimID[test_index]

            logreg = linear_model.LogisticRegression(max_iter=max_iter,solver='sag',multi_class='multinomial')

            logreg.fit(X_train, np.ravel(y_train))

            pred[test_index,ind] = logreg.predict(X_test)

        perc_correct[ind] = metrics.accuracy_score(StimID, pred[:,ind])
        # confusion_matrix = metrics.confusion_matrix(StimID, pred[:,ind,n])

    return(perc_correct, pred)

In [ ]:
# Compute logistic regression

# pred = np.zeros([numTrials,n_iter,numX])
# perc_correct = np.zeros([n_iter,numX])
# for n, N in enumerate(number_of_neurons):
#     perc_correct[:,n], pred[:,:,n] = computeLogReg(N)

e = Executor()
Futures = e.map(computeLogReg, number_of_neurons)
out = e.gather(Futures)
e.close()
pred = np.zeros([numTrials,n_iter,numX])
perc_correct = np.zeros([n_iter,numX])
for n, N in enumerate(number_of_neurons):
    perc_correct[:,n] = out[n][0]
    pred[:,:,n] = out[n][1]
    
    
print('Done!')

In [ ]:
h5f = h5py.File(filebase + '.h5', 'w')
h5f.create_dataset('/perc_correct', data=perc_correct)
h5f.create_dataset('/pred', data=pred)
h5f.create_dataset('/number_of_neurons', data=number_of_neurons)
h5f.close()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
ax.errorbar(number_of_neurons,np.mean(perc_correct,axis=0),yerr=1.96*np.std(perc_correct,axis=0))
ax.set_ylabel('Percent Correct')
ax.set_xlabel('# of Neurons')

In [ ]:
x = np.ceil(np.sqrt(numX)).astype('int')
y = np.ceil(numX/x).astype('int')
fig, axs = plt.subplots(y,x,sharex=True,sharey=True,figsize=(10,10))
for ax, n in zip(axs.flat,range(numX)):
    confusion_matrix = metrics.confusion_matrix(np.repeat(StimID,n_iter), np.ravel(pred[:,:,n]))
    ax.imshow(confusion_matrix)
    ax.set_title('%d' % number_of_neurons[n])